# Import libraries

In [ ]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver import ActionChains
import re
import time
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
!pip install --upgrade google-api-python-client
!pip install oauth2client

In [ ]:
import httplib2
import os
from oauth2client import client, tools
from oauth2client import file as oa2file
import base64
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from apiclient import errors, discovery
from os.path import basename
from email.mime.application import MIMEApplication

In [ ]:
def pages(courses, base_url="https://classes.usc.edu/term-20221/course/"):
    """
    Page generator that returns webpage urls
    :param base_url: str
    :return: str
    """

    for course in courses:
        new_url = f"{base_url}/{course.lower().replace(' ', '-')}"
        yield new_url, course, courses[course]                                           # yields new url

# init incognito browser

> download latest compatible chromedriver version before running this cell

In [ ]:
def create_browser_instance():
    chrome_driver_path = r"chromedriver_win32/chromedriver.exe"     # chrome_driver path from present directory
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")                              # get a headless chrome instance
    options.add_argument("--incognito")                             # to start a fresh session without prior cookies
    browser = webdriver.Chrome(executable_path=chrome_driver_path, options=options)
    return browser

browser = create_browser_instance()

# Input requirement:

Find the courses you want to apply to and save them to a courses.txt in the following format.
Department Course_number: Name of the course (x.x units)
> [(x.x units) part can be excluded]

For e.g:

> CSCI 561: Foundations of Artificial Intelligence (4.0 units) <br>
> CSCI 566: Deep Learning and Its Applications (4.0 units)

I have attached an example courses.txt for reference

# Setup client_secret.json file through [Python OAuth2](https://stackoverflow.com/questions/37201250/sending-email-via-gmail-python)

## Step 1:
### Note: in step d, select Web application in 'Application type' since Jupyter-notebook is a web application
<img src="https://i.stack.imgur.com/ICIXt.png" width="600" height="300" />

# Create course dictionary

In [ ]:
courses = {}        # contains course names
store_dets = []     # contains other course related details
if os.path.exists('courses.txt'):
    with open('courses.txt', "r") as f:
        for i in f.readlines():
            match = re.match(r'(\w+\s\d+\w*): (.+) (\(.+\))', i)
            if match:
                courses[match.group(1)] = match.group(2)
else:
    print(f"courses.txt doesn't exist, please create and fill it with required info")

# Get all relevant info

In [ ]:
year = input("Year->") # default: 2022
term = input("1: Spring; 2: Summer; 3: Fall;\n----->") # default: 1
for page, course, name in pages(courses, base_url=f"https://classes.usc.edu/term-{year}{term}/course"):
    print(page)
    browser.get(page)                                                      # load page into browser
    source = browser.page_source
    soup = bs(source, "html.parser").find('tbody')
    for section in soup.findAll("tr"):
        if section.attrs['class'] == ["headers"]:
            continue
        childrens = list(section.children)
        lecture_section = childrens[2]
        if lecture_section.text.lower() == "lecture":
            instructor = childrens[6]
            found = instructor.find('a')
            browser.get("https://uscdirectory.usc.edu/web/directory/faculty-staff/")
            element_to_search = browser.find_element(By.NAME, "q")
            action = ActionChains(browser)
            action.click(element_to_search)
            action.send_keys(instructor.text)
            action.perform()
            click_button = browser.find_element(By.TAG_NAME, "button")
            action.click(click_button)
            action.perform()
            time.sleep(5)
            source = browser.page_source
            all_emails = re.findall(r'[-\w\.]+@[\w+.]+\w+', browser.page_source)
            store_dets.append((course, name, instructor.text, all_emails[-1] if all_emails != [] else None,found.attrs['href'] if found else None))
browser.close()

# Load into a PD DataFrame

In [ ]:
info = pd.DataFrame(set(store_dets), columns=['syllable', 'course', 'professor', 'email', 'links']).sort_values('syllable')
info.to_csv("details.csv", index=False)

### NOTE
> Please check if the emails correspond to professor names from the 'professor' section

In [ ]:
info

# [Google copy-pasta for sending email through python](https://developers.google.com/gmail/api/guides/sending)

In [ ]:
SCOPES = 'https://www.googleapis.com/auth/gmail.send'
CLIENT_SECRET_FILE = "client_secret.json"
APPLICATION_NAME = 'Gmail API Python Send Email'

def get_credentials():
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir, 'gmail-python-email-send.json')
    store = oa2file.Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        credentials = tools.run_flow(flow, store)
        print('Storing credentials to ' + credential_path)
    return credentials

def SendMessage(sender, to, subject, msgHtml, filename):
    # filename -> name of file to be attached
    credentials = get_credentials()
    http = credentials.authorize(httplib2.Http())
    service = discovery.build('gmail', 'v1', http=http)
    message1 = CreateMessage(sender, to, subject, msgHtml, filename=filename)
    SendMessageInternal(service, "me", message1)

def SendMessageInternal(service, user_id, message):
    try:
        message = (service.users().messages().send(userId=user_id, body=message).execute())
        print('Message sent - id: %s' % message['id'])
        return message
    except errors.HttpError as error:
        print('An error occurred: %s' % error)

def CreateMessage(sender, to, subject, msgHtml, filename):
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = to
            
    with open(filename, "rb") as f:
        part = MIMEApplication(f.read(),Name=basename(filename))
        # After the file is closed
    part['Content-Disposition'] = 'attachment; filename="%s"' % basename(filename)
    msg.attach(part)
    
    msg.attach(MIMEText(msgHtml, 'html'))
    raw = base64.urlsafe_b64encode(msg.as_bytes())
    raw = raw.decode()
    body = {'raw': raw}
    return body

# This is what we are here for

- Please have your resume/C.V. in the same path as this notebook <br>
- Replace "your_email" with the your actual email on line 9 <br>
- Replace "your_resume.pdf" with the path/name of your file
- Modify the msgHtml for custom email body
- Modify the subject variable 

In [ ]:
def print_automate():
    if os.path.exists("details.csv"):
        details = pd.read_csv("details.csv")
        msgHtml = """{0} example {1}""" # Email body, 0 - professor's name, 1 - course name
        for i in range(5):
            current = details.iloc[i]
            course = f"{current['syllable']}: {current['course']}"
            to = current['email'] # email address of the professor
            sender = "your_email" # your email
            prof_name = current['professor']
            subject = "Sample" # email subject
            SendMessage(sender, to, subject, msgHtml.format(prof_name,course), "your_resume.pdf") # name corresponds to filename
            print(current['email'])
    else:
        print(f"details.csv file doesn't exist")

# RUN!

In [ ]:
if __name__ == '__main__':
    print_automate()